The general idea behind boosting algorithms is to transform weak learners into strong learners. A weak learner is hardly better than random guessing. But there is a purpose behind the weak start. Building on this general idea, boosting works by focusing on iterative error correction, not by establishing a strong baseline model. If the base model is too strong, the learning process is necessarily limited, thereby undermining the general strategy behind boosting models.

Weak learners are transformed into strong learners through hundreds of iterations. Boosting has been one of the best general machine learning strategies in terms of producing optimal results. 

### Gradient Boosting

While gradient boosting also adjusts based on incorrect predictions (similar to AdaBoost), it takes this idea one step further: gradient boosting fits each new tree entirely based on the errors of the previous tree's predictions. That is, for each new tree, gradient boosting looks at the mistakes and then builds a new tree completely around these mistakes. The new tree doesn't care about the predictions that are already correct.
Gradient boosting computes the residuals of each tree's predictions and sums all the residuals to score the model.
A gradient boosting model is built from scratch by training new trees on the errors of the previous trees. 
We continue with the bike rentals dataset to compare new models with the previous models:

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load the bike rental dataset:
df_bikes = pd.read_csv('bike_rentals_cleaned.csv')
df_bikes.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1.0,0.0,1,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446,985
1,2,1.0,0.0,1,0.0,0.0,0.0,2,0.363478,0.353739,0.696087,0.248539,801
2,3,1.0,0.0,1,0.0,1.0,1.0,1,0.196364,0.189405,0.437273,0.248309,1349
3,4,1.0,0.0,1,0.0,2.0,1.0,1,0.200000,0.212122,0.590435,0.160296,1562
4,5,1.0,0.0,1,0.0,3.0,1.0,1,0.226957,0.229270,0.436957,0.186900,1600


#### Building a gradient boosting model from scratch

In [4]:
# split the data into X and y. Then, split X and y into training and test sets:
X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

In [4]:
# Fit the data to the decision tree:  The initial decision tree, called a base learner, should not be fine-tuned for accuracy. 
# We want a model that focuses on learning from errors, not a model that relies heavily on the base learner.
# Initialize a decision tree with max_depth=2 and fit it on the training set as tree_1, since it's the first tree in our ensemble:
from sklearn.tree import DecisionTreeRegressor
tree_1 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_1.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [5]:
#Make predictions with the training set: Instead of making predictions with the test set, predictions in gradient boosting are initially made with the training set. 
#To compute the residuals, we need to compare the predictions while still in the training phase. The test phase of the model build comes at the end, after all the trees have been constructed. 
#The predictions of the training set for the first round are obtained by adding the predict method to tree_1 with X_train as the input:
y_train_pred = tree_1.predict(X_train)

In [6]:
# Compute the residuals: The residuals are the differences between the predictions and the target column. 
# The predictions of X_train, defined here as y_train_pred, are subtracted from y_train, the target column, to compute the residuals:
y2_train = y_train - y_train_pred
# The residuals are defined as y2_train because they are the new target column for the next tree.

In [7]:
# Fit the new tree on the residuals: Fitting a new tree on the residuals is different than fitting a model on the training set. 
#The primary difference is in the predictions. In the bike rentals dataset, when fitting a new tree on the residuals, we should progressively get smaller numbers.
#Initialize a new tree and fit it on X_train and the residuals, y2_train:
tree_2 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_2.fit(X_train, y2_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [8]:
# Repeat steps 2-4: As the process continues, the residuals should gradually approach 0 from the positive and negative direction. 
#The iterations continue for the number of estimators, n_estimators.
# Let's repeat the process for a third tree as follows:
y2_train_pred = tree_2.predict(X_train)
y3_train = y2_train - y2_train_pred
tree_3 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_3.fit(X_train, y3_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [9]:
# As a learning purpose, stop the process here.
# Sum the results: Summing the results requires making predictions for each tree with the test set as follows:
y1_pred = tree_1.predict(X_test)
y2_pred = tree_2.predict(X_test)
y3_pred = tree_3.predict(X_test)

In [10]:
y_pred = y1_pred + y2_pred + y3_pred

In [11]:
# Lastly, let's compute the mean squared error (MSE) to obtain the results as follows:
from sklearn.metrics import mean_squared_error as MSE
MSE(y_test, y_pred)**0.5

911.0479538776444

#### Building a gradient boosting model in scikit-learn

Use GradientBoostingRegressor with hyperparameter adjustments. The advantage of using GradientBoostingRegressor is that it's much faster to build and easier to implement:

In [7]:
# import the regressor from the sklearn.ensemble library:
from sklearn.ensemble import GradientBoostingRegressor

In [13]:
# To obtain the same results, it's essential to match max_depth=2 and random_state=2. 
# Since there are only three trees, we must have n_estimators=3. 
# Finally, we must set the learning_rate=1.0 hyperparameter.
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=3, random_state=2, learning_rate=1.0)

In [14]:
# the model has been initialized, it can be fit on the training data and scored against the test data:
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

911.0479538776439

The result is the same to 11 decimal places.
Recall that the point of gradient boosting is to build a model with enough trees to transform a weak learner into a strong learner. This is easily done by changing n_estimators, the number of iterations, to a much larger number.

In [15]:
# Let's build and score a gradient boosting regressor with 30 estimators:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=30, random_state=2, learning_rate=1.0)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

857.1072323426944

In [16]:
# when n_estimators = 300:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2, learning_rate=1.0)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

936.3617413678853

In [17]:
# remove parameter learning_rate:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

653.7456840231495

### Modifying GB Hyperparameters

learning_rate, also known as the shrinkage, shrinks the contribution of individual trees so that no tree has too much influence when building the model. If an entire ensemble is built from the errors of one base learner, without careful adjustment of hyperparameters, early trees in the model can have too much influence on subsequent development. learning_rate limits the influence of individual trees. Generally speaking, as n_estimators, the number of trees, goes up, learning_rate should go down.
learning_rate ranges from 0 to 1. A learning_rate value of 1 means that no adjustments are made. The default value of 0.1 means that the tree's influence is weighted at 10%.

In [19]:
# build and score a new GradientBoostingRegressor to see how the scores compare:
learning_rate_values = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5, 1.0]

for value in learning_rate_values:
    gbr = GradientBoostingRegressor(max_depth=2,   n_estimators=300, random_state=2, learning_rate=value)
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    rmse = MSE(y_test, y_pred)**0.5
    print('Learning Rate:', value, ', Score:', rmse)

Learning Rate: 0.001 , Score: 1633.0261400367258
Learning Rate: 0.01 , Score: 831.5430182728547
Learning Rate: 0.05 , Score: 685.0192988749717
Learning Rate: 0.1 , Score: 653.7456840231495
Learning Rate: 0.15 , Score: 687.666134269379
Learning Rate: 0.2 , Score: 664.312804425697
Learning Rate: 0.3 , Score: 689.4190385930236
Learning Rate: 0.5 , Score: 693.8856905068778
Learning Rate: 1.0 , Score: 936.3617413678853


##### the default learning_rate value of 0.1 gives the best score for 300 trees.It's important to tune learning_rate and n_estimators together.

Base learner
The initial decision tree in the gradient boosting regressor is called the base learner because it's at the base of the ensemble. It's the first learner in the process. It is a weak learner transforming into a strong learner.

In [20]:
#It's possible to tune base learners for gains in accuracy.
# select a max_depth value of 1, 2, 3, or 4 and compare results as follows:
depths = [None, 1, 2, 3, 4]

for depth in depths:
    gbr = GradientBoostingRegressor(max_depth=depth, n_estimators=300, random_state=2)
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    rmse = MSE(y_test, y_pred)**0.5
    print('Max Depth:', depth, ', Score:', rmse)

Max Depth: None , Score: 869.2788645118395
Max Depth: 1 , Score: 707.8261886858736
Max Depth: 2 , Score: 653.7456840231495
Max Depth: 3 , Score: 646.4045923317708
Max Depth: 4 , Score: 663.048387855927


subsample 
is a subset of samples. Since samples are the rows, a subset of rows means that all rows may not be included when building each tree. By changing subsample from 1.0 to a smaller decimal, trees only select that percentage of samples during the build phase. For example, subsample=0.8 would select 80% of samples for each tree.

In [21]:
samples = [1, 0.9, 0.8, 0.7, 0.6, 0.5]

for sample in samples:
    gbr = GradientBoostingRegressor(max_depth=3, n_estimators=300, subsample=sample, random_state=2)
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    rmse = MSE(y_test, y_pred)**0.5
    print('Subsample:', sample, ', Score:', rmse)

Subsample: 1 , Score: 646.4045923317708
Subsample: 0.9 , Score: 620.1819001443569
Subsample: 0.8 , Score: 617.2355650565677
Subsample: 0.7 , Score: 612.9879156983139
Subsample: 0.6 , Score: 622.6385116402317
Subsample: 0.5 , Score: 626.9974073227554


RandomizedSearchCV
Our preliminary analysis indicated that a grid search centered around max_depth=3, subsample=0.7, n_estimators=300, and learning_rate = 0.1 is a good place to start. As n_estimators goes up, learning_rate should go down..RandomizedSearchCV is preferred over GridSearchCV when there is too many combinations of hyperparameters. It will speed up computations.

In [5]:
# starting point:
params={'subsample':[0.65, 0.7, 0.75],
        'n_estimators':[300, 500, 1000],
         'learning_rate':[0.05, 0.075, 0.1]}

In [8]:
#import RandomizedSearchCV and initialize a gradient boosting model:
from sklearn.model_selection import RandomizedSearchCV

gbr = GradientBoostingRegressor(max_depth=3, random_state=2)

In [9]:
# Initialize RandomizedSearchCV with gbr and params as inputs in addition to the number of iterations, the scoring, and the number of folds. 
# Recall that n_jobs=-1 may speed up computations and random_state=2 ensures the consistency of results:
rand_reg = RandomizedSearchCV(gbr, params, n_iter=10, scoring='neg_mean_squared_error', cv=5, n_jobs=-1, random_state=2)

In [11]:
# fit the model on the training set and obtain the best parameters and scores:
from sklearn.metrics import mean_squared_error as MSE

rand_reg.fit(X_train, y_train)
best_model = rand_reg.best_estimator_
best_params = rand_reg.best_params_

print("Best params:", best_params)
best_score = np.sqrt(-rand_reg.best_score_)
print("Training score: {:.3f}".format(best_score))

y_pred = best_model.predict(X_test)
rmse_test = MSE(y_test, y_pred)**0.5
print('Test set score: {:.3f}'.format(rmse_test))

Best params: {'subsample': 0.65, 'n_estimators': 300, 'learning_rate': 0.05}
Training score: 636.200
Test set score: 625.985


In [12]:
# after experimentations, a better model is possible:
gbr = GradientBoostingRegressor(max_depth=3, n_estimators=1600, subsample=0.75, learning_rate=0.02, random_state=2)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

596.9544588974487

XGBoost
XGBoost is an advanced version of gradient boosting with the same general structure, it transforms weak learners into strong learners by summing the residuals of trees.
The only difference in hyperparameters from the last section is that XGBoost refers to learning_rate as eta.

In [13]:
# intialize the model:
from xgboost import XGBRegressor

xg_reg = XGBRegressor(max_depth=3, n_estimators=1600, eta=0.02, subsample=0.75, random_state=2)
xg_reg.fit(X_train, y_train)
y_pred = xg_reg.predict(X_test)
MSE(y_test, y_pred)**0.5

584.3395337495713

##### XGBoost is preferred over gradient boosting in general because it consistently delivers better results, and because it's faster.

### Gradient boosting versus XGBoost - exoplanet example

We examine exoplanets over time. The dataset has 5,087 rows and 3,189 columns that record light flux at different times of a star's life cycle. Multiplying columns and rows together results in 1.5 million data points. Using a baseline of 100 trees, we need 150 million data points to build a model.
Each row is an individual star and the columns reveal different light patterns over time. In addition to light patterns, an exoplanet column is labeled 2 if the star hosts an exoplanet; otherwise, it is labeled 1.

In [14]:
# # Load the exoplanet dataset:
df = pd.read_csv('exoplanets.csv')
df.head()

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,93.85,83.81,20.10,-26.98,-39.56,-124.71,-135.18,-96.27,-79.89,...,-78.07,-102.15,-102.15,25.13,48.57,92.54,39.32,61.42,5.08,-39.54
1,2,-38.88,-33.83,-58.54,-40.09,-79.31,-72.81,-86.55,-85.33,-83.97,...,-3.28,-32.21,-32.21,-24.89,-4.86,0.76,-11.70,6.46,16.00,19.93
2,2,532.64,535.92,513.73,496.92,456.45,466.00,464.50,486.39,436.56,...,-71.69,13.31,13.31,-29.89,-20.88,5.06,-11.80,-28.91,-70.02,-96.67
3,2,326.52,347.39,302.35,298.13,317.74,312.70,322.33,311.31,312.42,...,5.71,-3.73,-3.73,30.05,20.03,-12.67,-8.77,-17.31,-17.35,13.98
4,2,-1107.21,-1112.59,-1118.95,-1095.10,-1057.55,-1034.48,-998.34,-1022.71,-989.57,...,-594.37,-401.66,-401.66,-357.24,-443.76,-438.54,-399.71,-384.65,-411.79,-510.54


In [15]:
# are all columns numerical?
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5087 entries, 0 to 5086
Columns: 3198 entries, LABEL to FLUX.3197
dtypes: float64(3197), int64(1)
memory usage: 124.1 MB


In [16]:
# number of null values?
df.isnull().sum().sum()

0

In [17]:
# split the data into training and test sets. 
#Note that the 0th column is the target column, y, and all other columns are the predictor columns, X:
X = df.iloc[:,1:]
y = df.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

#### Building gradient boosting classifiers

In [18]:
# import GradientBoostingClassifer and XGBClassifier in addition to accuracy_score so that we may compare both models:
from sklearn.ensemble import GradientBoostingClassifier

from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score

#### Compare models using a timer

In [19]:
import time

In [20]:
# try an example:
start = time.time()
df.info()
end = time.time()
elapsed = end - start

print('\nRun Time: ' + str(elapsed) + ' seconds.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5087 entries, 0 to 5086
Columns: 3198 entries, LABEL to FLUX.3197
dtypes: float64(3197), int64(1)
memory usage: 124.1 MB

Run Time: 0.014196634292602539 seconds.


####  Compare GradientBoostingClassifier and XGBoostClassifier with the exoplanet dataset for its speed

In [21]:
#GradientBoostingClassifier:
start = time.time()
gbr = GradientBoostingClassifier(n_estimators=100, max_depth=2, random_state=2)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
score = accuracy_score(y_pred, y_test)
print('Score: ' + str(score))
end = time.time()

elapsed = end - start
print('\nRun Time: ' + str(elapsed) + ' seconds')

Score: 0.9874213836477987

Run Time: 162.29657530784607 seconds


In [22]:
# XGBClassifier:
start = time.time()
xg_reg = XGBClassifier(n_estimators=100, max_depth=2, random_state=2)
xg_reg.fit(X_train, y_train)
y_pred = xg_reg.predict(X_test)
score = accuracy_score(y_pred, y_test)

print('Score: ' + str(score))
end = time.time()

elapsed = end - start
print('Run Time: ' + str(elapsed) + ' seconds')

[05:00:39] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Score: 0.9913522012578616
Run Time: 6.5532145500183105 seconds
